# Evidence QA with BERT on Colab TPU

This notebook mirrors the local Hydra/Optuna/MLflow setup so you can launch extractive QA experiments from Google Colab.

⚙️ **Prerequisites**

- Switch the Colab runtime to **TPU** (Runtime → Change runtime type → Hardware accelerator → TPU).
- (Optional) Mount Google Drive if you want the MLflow/Optuna SQLite databases and artifacts to persist across sessions.

In [ ]:
#@title Clone your repository (update REPO_URL if needed)
import os
from pathlib import Path

REPO_URL = "https://github.com/your-account/DataAug_Evidence_BERT.git"  # @param {type:"string"}
TARGET_DIR = Path("/content/DataAug_Evidence_BERT")

if not TARGET_DIR.exists():
    !git clone "{REPO_URL}" "{TARGET_DIR}"
os.chdir(TARGET_DIR)
print("Current working directory:", Path.cwd())
print("Repository content:", os.listdir())

In [ ]:
#@title Install dependencies for TPU training
%%bash
pip install -q torch~=2.1.0 torchvision~=0.16.0 torchaudio~=2.1.0 torch_xla~=2.1.0 \
  -f https://storage.googleapis.com/tpu-pytorch/wheels/tpuvm/torch_xla-2.1-cp310-cp310-linux_x86_64.whl
pip install -q -r requirements.txt --no-deps

In [ ]:
#@title Configure experiment tracking paths
import os
from pathlib import Path

project_root = Path.cwd()
mlflow_db = project_root / "mlflow.db"
mlruns_dir = project_root / "mlruns"
optuna_db = project_root / "optuna.db"

os.environ["MLFLOW_TRACKING_URI"] = f"sqlite:///{mlflow_db}"
os.environ["OPTUNA_STORAGE"] = f"sqlite:///{optuna_db}"
mlruns_dir.mkdir(exist_ok=True)

print("MLflow tracking URI:", os.environ["MLFLOW_TRACKING_URI"])
print("Optuna storage URI:", os.environ["OPTUNA_STORAGE"])

In [ ]:
#@title Validate TPU availability
import torch_xla.core.xla_model as xm

device = xm.xla_device()
print("TPU device: ", device)
print("TPU cores: ", xm.xrt_world_size())

In [ ]:
#@title Launch a training run (Hydra overrides supported)
overrides = [
    "training.tpu_num_cores=8",
    "training.per_device_train_batch_size=16",
    "training.gradient_accumulation_steps=1",
    "model.head.num_layers=2",
]
overrides_str = " ".join(overrides)

!python -m src.cli train $overrides_str

In [ ]:
#@title Run Optuna search (set n_trials small for TPU runtime limits)
overrides = [
    "training.tpu_num_cores=8",
    "hpo.n_trials=5",
    "hpo.timeout=7200",
]
overrides_str = " ".join(overrides)

!python -m src.cli hpo $overrides_str

📈 **Review results**

- Point an MLflow UI at `/content/DataAug_Evidence_BERT/mlruns` to browse experiment artifacts.
- Inspect `optuna.db` with `optuna-dashboard` or `optuna.study.load_study` to analyze the HPO trials.